
# **LAP 4 : MODEL BUILDING !!**

---



## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">


[LOAD DATA FROM DRIVE INTO DATAFRAME](#1)<br>
[CREATE DATASET FROM 'FEATURE SELECTED' LIST](#2)<br>

[MODEL DEFINITIONS](#3)<br>
  A. BUILDING LOGISTIC REGRESSION MODEL<br>
  B. BUILDING DECISION TREE CLASSIFIER<br>
  C. BUILDING RANDOMFORESTCLASSIFIER<br>
  D. BUILDING ENSEMBLE WITH EXTRATREESCLASSIFIER<br>
  E. BUILDING ADABOOSTCLASSIFIER WITH DTC<br>
  F. BUILDING GRADIENTBOOSTINGCLASSIFIER<br>

[SUMMARY MODEL REPORT](#4)<br>
[SUMMARY MODEL REPORT ON OUT-OF-SAMPLE DATA](#5)<br>
[ESTIMATED MONETARY IMPACT](#6)<br>
[CONCLUSION](#7)<br>

</div>


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###           IMPORT LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import seaborn as sns
import pickle as pkl
sns.set_style('whitegrid')
get_ipython().run_line_magic('matplotlib', 'inline')

from sklearn import metrics
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import cohen_kappa_score as kappa
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

pd.set_option('display.max_columns', 200)
pd.set_option('max_colwidth', 30)
pd.set_option('display.max_rows', 100)

plt.style.use('default')
plt.rcParams['figure.figsize'] = (8,5) # (12,8)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

##      LOAD DATA FROM DRIVE INTO DATAFRAME       <a id="1"></a>

In [ ]:
path = '/content/drive/MyDrive/LAP/'

Xt_orig = pd.read_csv(path + 'Xtrn.csv', index_col=False)
Yt_orig = pd.read_csv(path + 'Ytrn.csv', index_col=False)
Xv_orig = pd.read_csv(path + 'Xval.csv', index_col=False)
Yv_orig = pd.read_csv(path + 'Yval.csv', index_col=False)
Xo_orig = pd.read_csv(path + 'Xout.csv', index_col=False)
Yo_orig = pd.read_csv(path + 'Yout.csv', index_col=False)
df_original_sub = pd.read_csv(path + 'submission.csv', index_col=False)

Yt=np.array(Yt_orig).ravel(); Yv=np.array(Yv_orig).ravel(); Yo=np.array(Yo_orig).ravel()

###               CHECK  DF  INTEGRITY   

In [ ]:
lstc = [Xt_orig,Yt_orig,Xv_orig,Yv_orig,Xo_orig,Yo_orig]
for i in lstc: print(i.shape," ", type(i)); i.index; i.head(2); i.tail(2)

(367812, 43)   <class 'pandas.core.frame.DataFrame'>


RangeIndex(start=0, stop=367812, step=1)

,loan_amnt,term,int_rate,grade,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,acc_now_delinq,monthly_inc,EMI,tot_bal,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_f,initial_list_status_w
0,-0.229203,1.612785,1.362588,1.743227,0.891014,1.220382,-0.358395,-0.766066,0.790200,-0.330564,0.021966,0.855631,0.541485,-0.09611,-0.061688,-0.812075,-0.716246,-0.784859,-0.001418,1.049376,-0.008854,-0.015467,-0.320697,-0.869447,-0.69943,-0.736398,1.458107,-0.10504,1.847288,-1.224085,-0.022466,-0.234284,-0.06402,-0.141479,-0.09759,-0.079166,-0.22726,-0.026379,-0.109093,-0.073548,-0.062272,-1.240716,1.240716
1,0.794387,1.612785,-0.461455,-0.589153,0.819845,0.929985,-0.358395,-0.766066,0.321438,-0.330564,1.336221,0.213880,-0.147537,-0.09611,-0.061688,0.501230,0.070543,0.513436,-0.001418,1.049376,-0.008854,-0.015467,-0.320697,-0.869447,-0.69943,-0.736398,1.458107,-0.10504,-0.541334,0.816937,-0.022466,-0.234284,-0.06402,-0.141479,-0.09759,-0.079166,-0.22726,-0.026379,-0.109093,-0.073548,-0.062272,0.805986,-0.805986


,loan_amnt,term,int_rate,grade,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,acc_now_delinq,monthly_inc,EMI,tot_bal,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_f,initial_list_status_w
367810,-1.347835,-0.620045,1.239918,0.965767,-0.626650,-0.499023,-0.358395,-0.766066,-0.444019,-0.330564,-0.815069,1.334017,-1.354886,-0.09611,-0.061688,-1.428468,-1.362552,-1.373782,-0.001418,-0.952947,-0.008854,-0.015467,-0.320697,1.150156,-0.699430,-0.736398,1.458107,-0.10504,-0.541334,0.816937,-0.022466,-0.234284,-0.06402,-0.141479,-0.09759,-0.079166,-0.22726,-0.026379,-0.109093,-0.073548,-0.062272,0.805986,-0.805986
367811,-1.397581,-0.620045,-0.153145,0.188307,0.866936,0.754121,-0.358395,-0.711716,1.078020,1.586462,-0.264392,0.972684,0.942692,-0.09611,-0.061688,-0.416902,-1.426262,-0.324398,-0.001418,-0.952947,-0.008854,-0.015467,-0.320697,1.150156,1.429735,-0.736398,-0.685821,-0.10504,-0.541334,0.816937,-0.022466,-0.234284,-0.06402,-0.141479,-0.09759,-0.079166,-0.22726,-0.026379,-0.109093,-0.073548,-0.062272,0.805986,-0.805986


(367812, 1)   <class 'pandas.core.frame.DataFrame'>


...

,loan_amnt,term,int_rate,grade,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,acc_now_delinq,monthly_inc,EMI,tot_bal,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_f,initial_list_status_w
211892,1.342340,-0.681059,-1.370307,-1.369549,-1.570813,-0.655210,-0.375873,-0.679436,0.014833,-0.380355,2.928803,1.657883,-0.819573,-0.13186,-0.068777,1.498597,2.138248,1.392151,-0.003072,1.060829,0.0,0.0,-0.351423,-0.850193,-0.646115,-0.850423,1.580831,-0.095549,1.754049,-1.188795,0.0,-0.252554,-0.056531,-0.137379,-0.101483,-0.077682,-0.225991,-0.022997,-0.086287,-0.075468,-0.002172,-0.679033,0.679033
211893,1.301753,1.468301,-1.010105,-0.580917,-1.570813,0.035659,-0.375873,-0.679436,0.916151,-0.380355,0.047745,-1.144802,0.367677,-0.13186,-0.068777,2.905749,0.498919,2.966345,-0.003072,-0.942659,0.0,0.0,-0.351423,1.176204,-0.646115,-0.850423,1.580831,-0.095549,-0.570110,0.841188,0.0,-0.252554,-0.056531,-0.137379,-0.101483,-0.077682,-0.225991,-0.022997,-0.086287,-0.075468,-0.002172,-0.679033,0.679033


(211894, 1)   <class 'pandas.core.frame.DataFrame'>


RangeIndex(start=0, stop=211894, step=1)

,default_ind
0,0
1,0


,default_ind
211892,0
211893,0


##      CREATE DATASET FROM 'FEATURE SELECTED' LIST       <a id="2"></a>

In [ ]:
# Import List of 'Selected Feature Set' created in Feature Seelection module.

with open(path + 'fsDict', "rb") as fp:   # Unpickling
   a = pkl.load(fp)

# Display the datasets and its features:
print(a)

'NOTE: Do NOT run these two cell if you want to model on the original dataset'

{'DS_Original': array(['loan_amnt', 'term', 'int_rate', 'grade', 'issue_d', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc',
       'collections_12_mths_ex_med', 'acc_now_delinq', 'monthly_inc',
       'EMI', 'tot_bal', 'home_ownership_ANY', 'home_ownership_MORTGAGE',
       'home_ownership_NONE', 'home_ownership_OTHER',
       'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified',
       'verification_status_Verified', 'purpose_car',
       'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy',
       'purpose_small_business', 'purpose_vacation', 'purpose_wedding',
       'initial_list_status_f', 'initial_list_status_w'], dtype=object), 'DS_RFE': ar

In [ ]:
# SUMMARY of the Datasets
for x, y in a.items(): print("Length of Data Set", x, "= ", len(y))

Length of Data Set DS_Original =  43
Length of Data Set DS_RFE =  21
Length of Data Set DS_UFS =  16
Length of Data Set DS_VT =  40
Length of Data Set DS_DT =  13
Length of Data Set DS_Ada =  14


In [ ]:
# Since some of the datasets are similar in structure we choose only few of them.
# We shall pick these datasets for modeling : DS_Original, DS_RFE, DS_Ada

# Keep the above datasets in 'a'
a_copy = a    # a = a_copy
for key in ('DS_UFS', 'DS_VT', 'DS_DT'):
    del a[key]
a

{'DS_Original': array(['loan_amnt', 'term', 'int_rate', 'grade', 'issue_d', 'dti',
        'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec',
        'revol_bal', 'revol_util', 'total_acc',
        'collections_12_mths_ex_med', 'acc_now_delinq', 'monthly_inc',
        'EMI', 'tot_bal', 'home_ownership_ANY', 'home_ownership_MORTGAGE',
        'home_ownership_NONE', 'home_ownership_OTHER',
        'home_ownership_OWN', 'home_ownership_RENT',
        'verification_status_Not Verified',
        'verification_status_Source Verified',
        'verification_status_Verified', 'purpose_car',
        'purpose_credit_card', 'purpose_debt_consolidation',
        'purpose_educational', 'purpose_home_improvement', 'purpose_house',
        'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
        'purpose_other', 'purpose_renewable_energy',
        'purpose_small_business', 'purpose_vacation', 'purpose_wedding',
        'initial_list_status_f', 'initial_list_status_w'], dtype=object

## **MODEL DEFINITIONS**        <a id="3"></a>

In [ ]:
# Global Variables
Y_pred_rep = pd.DataFrame()
classifiersLst = []

###           A. BUILDING LOGISTIC REGRESSION MODEL

In [ ]:
from sklearn.linear_model import LogisticRegression

def algo_LgR(DS, Xt_LgR, Xv_LgR):
    #               CREATE MODEL INSTANCE
    classifierLg = LogisticRegression(solver='lbfgs', random_state=10)

    #               TRAIN MODEL WITH TRAIN SET
    classifierLg.fit(Xt_LgR, Yt)

    #               PREDICT WITH VALIDATION SET
    Y_pred_Lg = classifierLg.predict(Xv_LgR)
    # Y_pred_rep = pd.DataFrame()
    Y_pred_rep['Logistic Regression'] = Y_pred_Lg
    # classifiersLst = []
    classifiersLst.append(('Logistic Regression', classifierLg, DS))

    #               EVALUATE THE MODEL USING CM, CR, ACC
    print("Confusion Matrix (Logistic Regression): \n",confusion_matrix(Yv, Y_pred_Lg))
    print("\n Classification report: \n",classification_report(Yv,Y_pred_Lg))
    print("Accuracy of the model: ",accuracy_score(Yv, Y_pred_Lg))


###           B. BUILDING DECISION TREE CLASSIFIER  

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def algo_DT(DS, Xt_DT, Xv_DT):
    #               FIT THE DATA
    model_DecisionTree = DecisionTreeClassifier(criterion='gini', random_state=10)
    # Fit the model on the data
    model_DecisionTree.fit(Xt_DT,Yt)

    #               PREDICT FROM THE MODEL
    Y_pred_DT = model_DecisionTree.predict(Xv_DT)
    Y_pred_rep['Decision Tree'] = Y_pred_DT
    print("Y and Y_Pred", list(zip(Yv,Y_pred_DT))[0:20])
    classifiersLst.append(('Decision Tree', model_DecisionTree, DS))

    #               EVALUATE TREE MODEL
    print("Confusion Matrix (DecisionTree): \n",confusion_matrix(Yv, Y_pred_DT))
    print("\n Classification report: \n",classification_report(Yv,Y_pred_DT))
    print("Accuracy of the model: ",accuracy_score(Yv, Y_pred_DT))


###           C. BUILDING RANDOMFORESTCLASSIFIER

In [ ]:
def algo_RF(DS, Xt_RF, Xv_RF):
    #               FIT MODEL WITH TRAIN SET
    from sklearn.ensemble import RandomForestClassifier
    model_RF = RandomForestClassifier(100, random_state=10)
    model_RF.fit(Xt_RF, Yt)

    #               PREDICT ON VALIDATION DATA
    Y_pred_RF = model_RF.predict(Xv_RF)
    Y_pred_rep['Random Forest'] = Y_pred_RF
    classifiersLst.append(('Random Forest', model_RF, DS))

    #               EVALUATE THE MODEL
    print("Confusion Matrix (RandomForest): \n",confusion_matrix(Yv, Y_pred_RF))
    print("\n Classification report: \n",classification_report(Yv,Y_pred_RF))
    print("Accuracy of the model: ",accuracy_score(Yv, Y_pred_RF))


###           D. BUILDING ENSEMBLE WITH EXTRATREESCLASSIFIER

In [ ]:
def algo_ET(DS, Xt_ET, Xv_ET):
    #               FIT MODEL WITH TRAIN SET
    from sklearn.ensemble import ExtraTreesClassifier
    modelXt = ExtraTreesClassifier(50, random_state=10)
    modelXt.fit(Xt_ET, Yt)

    #               PREDICT ON VALIDATION DATA
    Y_pred_Xt = modelXt.predict(Xv_ET)
    Y_pred_rep['Extra Tree'] = Y_pred_Xt
    classifiersLst.append(('Extra Tree', modelXt, DS))

    #               EVALUATE THE MODEL
    print("Confusion Matrix (ExtraTrees): \n",confusion_matrix(Yv, Y_pred_Xt))
    print("\n Classification report: \n",classification_report(Yv,Y_pred_Xt))
    print("Accuracy of the model: ",accuracy_score(Yv, Y_pred_Xt))


###           E. BUILDING ADABOOSTCLASSIFIER WITH DTC

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def algo_Ada(DS, Xt_Ada, Xv_Ada):
    from sklearn.ensemble import AdaBoostClassifier

    #               BASE_ESTIMATOR = DTC
    model_ABdt = AdaBoostClassifier(estimator= DecisionTreeClassifier(), n_estimators=100, random_state=10)

    #               FIT MODEL WITH TRAIN SET
    model_ABdt.fit(Xt_Ada, Yt)

    #               PREDICT ON VALIDATION DATA
    Y_pred_ABdt = model_ABdt.predict(Xv_Ada)
    Y_pred_rep['AdaBoost+DTC'] = Y_pred_ABdt
    classifiersLst.append(('AdaBoost+DTC', model_ABdt, DS))

    #               EVALUATE THE MODEL
    print("Confusion Matrix (AdaBoost): \n",confusion_matrix(Yv, Y_pred_ABdt))
    print("\n Classification report: \n",classification_report(Yv,Y_pred_ABdt))
    print("Accuracy of the model: ",accuracy_score(Yv, Y_pred_ABdt))


###        F. BUILDING  GRADIENTBOOSTINGCLASSIFIER  

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

def algo_GB(DS, Xt_GB, Xv_GB):
    #               FIT MODEL WITH TRAIN SET
    model_GB = GradientBoostingClassifier( n_estimators=100, random_state=10)
    model_GB.fit(Xt_GB, Yt)

    #               PREDICT ON VALIDATION DATA
    Y_pred_GB = model_GB.predict(Xv_GB)
    Y_pred_rep['Gradient Boost'] = Y_pred_GB
    classifiersLst.append(('Gradient Boost', model_GB, DS))

    #               EVALUATE THE GBC MODEL
    print("Confusion Matrix (GradientBoosting): \n",confusion_matrix(Yv, Y_pred_GB))
    print("\n Classification report: \n",classification_report(Yv,Y_pred_GB))
    print("Accuracy of the model: ",accuracy_score(Yv, Y_pred_GB))


###    COMPARISON OF VARIOUS MODEL

In [ ]:
cols = ['DataSet', 'Model', 'Accuracy Score', 'ROC Score', 'Precision Score', 'Recall Score', 'F1 Score', \
        'Youden J Score', 'MCC', 'Kappa Score']

models_report = pd.DataFrame(columns = cols)

def model_compare(dataSet):
    global models_report
    for i, value in enumerate(Y_pred_rep.columns):
        tmp1 = pd.Series({'DataSet': dataSet, \
                    'Model': Y_pred_rep.columns[i], \
                    'Accuracy Score': metrics.accuracy_score(Yv, Y_pred_rep.iloc[:,i]), \
                    'ROC Score' : metrics.roc_auc_score(Yv, Y_pred_rep.iloc[:,i]), \
                    'Precision Score': metrics.precision_score(Yv, Y_pred_rep.iloc[:,i]), \
                    'Recall Score': metrics.recall_score(Yv, Y_pred_rep.iloc[:,i]), \
                    'F1 Score' : metrics.f1_score(Yv, Y_pred_rep.iloc[:,i]), \
                    'Youden J Score' : metrics.balanced_accuracy_score(Yv, Y_pred_rep.iloc[:,i]), \
                    'MCC': metrics.matthews_corrcoef(Yv, Y_pred_rep.iloc[:,i]), \
                    'Kappa Score':metrics.cohen_kappa_score(Yv, Y_pred_rep.iloc[:,i])})
        # models_report = models_report.append(tmp1, ignore_index = True)
        models_report = pd.concat([models_report, tmp1.to_frame().T], ignore_index=True)


##  **MODEL BUILDING**

In [ ]:
# Run Models for each DataSet
for key, val in a.items():
    Xt = Xt_orig[val]; Xv = Xv_orig[val]; Xo = Xo_orig[val]

    Y_pred_rep = pd.DataFrame()
    print(">>>>>>>>>>> For Data Set : ", key, " <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<" )
    algo_LgR(key, Xt, Xv)
    algo_DT(key, Xt, Xv)
    algo_RF(key, Xt, Xv)
    algo_ET(key, Xt, Xv)
    algo_Ada(key, Xt, Xv)
    algo_GB(key, Xt, Xv)
    model_compare(key)


>>>>>>>>>>> For Data Set :  DS_Original  <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Confusion Matrix (Logistic Regression): 
 [[62568 29325]
 [ 2500  5121]]

 Classification report: 
               precision    recall  f1-score   support

           0       0.96      0.68      0.80     91893
           1       0.15      0.67      0.24      7621

    accuracy                           0.68     99514
   macro avg       0.56      0.68      0.52     99514
weighted avg       0.90      0.68      0.75     99514

Accuracy of the model:  0.6801957513515686
Y and Y_Pred [(1, 0), (0, 0), (0, 1), (0, 1), (0, 0), (1, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 1), (0, 0), (0, 1), (1, 0), (0, 0), (0, 0), (0, 0), (1, 0), (0, 0), (0, 1)]
Confusion Matrix (DecisionTree): 
 [[78122 13771]
 [ 5583  2038]]

 Classification report: 
               precision    recall  f1-score   support

           0       0.93      0.85      0.89     91893
           1       0.13      0.27      0.17      7621

    accuracy              

## SUMMARY MODEL REPORT       <a id="4"></a>

In [ ]:
models_report

,DataSet,Model,Accuracy Score,ROC Score,Precision Score,Recall Score,F1 Score,Youden J Score,MCC,Kappa Score
0,DS_Original,Logistic Regression,0.680196,0.676419,0.148667,0.671959,0.243469,0.676419,0.197228,0.134981
1,DS_Original,Decision Tree,0.805515,0.55878,0.128914,0.267419,0.173965,0.55878,0.085522,0.078759
2,DS_Original,Random Forest,0.879886,0.571012,0.210273,0.206272,0.208253,0.571012,0.143281,0.143273
3,DS_Original,Extra Tree,0.873495,0.571342,0.198471,0.214539,0.206192,0.571342,0.137699,0.137576
4,DS_Original,AdaBoost+DTC,0.806389,0.560878,0.130895,0.270962,0.176518,0.560878,0.08865,0.081679
5,DS_Original,Gradient Boost,0.833441,0.618124,0.191241,0.363863,0.250712,0.618124,0.178067,0.16709
6,DS_RFE,Logistic Regression,0.680005,0.676316,0.148586,0.671959,0.243359,0.676316,0.197087,0.134843
7,DS_RFE,Decision Tree,0.805927,0.55443,0.12564,0.257447,0.168869,0.55443,0.07959,0.073503
8,DS_RFE,Random Forest,0.885413,0.566062,0.216151,0.188952,0.201638,0.566062,0.140583,0.140215
9,DS_RFE,Extra Tree,0.866883,0.575583,0.19277,0.231597,0.210407,0.575583,0.139081,0.138386


###    FIND THE BEST MODEL  

In [ ]:
classLst = []
numx1 = models_report['ROC Score'].max()
classLst.append(models_report[models_report['ROC Score'] == numx1].index.values)
numx2 = models_report['Precision Score'].max()
classLst.append(models_report[models_report['Precision Score'] == numx2].index.values)
numx3 = models_report['Recall Score'].max()
classLst.append(models_report[models_report['Recall Score'] == numx3].index.values)
numx4 = models_report['F1 Score'].max()
classLst.append(models_report[models_report['F1 Score'] == numx4].index.values)

uclassLst = np.unique(classLst)

ds1 = models_report.iloc[classLst[0][0],0]
ds2 = models_report.iloc[classLst[1][0],0]
ds3 = models_report.iloc[classLst[2][0],0]
ds4 = models_report.iloc[classLst[3][0],0]

mod1 = models_report.iloc[classLst[0][0],1]
mod2 = models_report.iloc[classLst[1][0],1]
mod3 = models_report.iloc[classLst[2][0],1]
mod4 = models_report.iloc[classLst[3][0],1]

print('The best ROC Score is : ', numx1, ' for dataSet : ', ds1, ' for model : ', mod1)
print('The best Precision Score is : ', numx2, ' for dataSet : ', ds2, ' for model : ', mod2)
print('The best Recall Score is : ', numx3, ' for dataSet : ', ds3, ' for model : ', mod3)
print('The best F1 Score is : ', numx4, ' for dataSet : ', ds4, ' for model : ', mod4)

The best ROC Score is :  0.6764189543582015  for dataSet :  DS_Original  for model :  Logistic Regression
The best Precision Score is :  0.2161513059141399  for dataSet :  DS_RFE  for model :  Random Forest
The best Recall Score is :  0.6723527096181603  for dataSet :  DS_Ada  for model :  Logistic Regression
The best F1 Score is :  0.25071199312870124  for dataSet :  DS_Original  for model :  Gradient Boost


#  

### PREDICTION ON OUT-SAMPLE DATA WITH THE BEST MODEL

In [ ]:
Y_pred_rep_out = pd.DataFrame()
cols = ['DataSet', 'Model', 'Accuracy Score', 'ROC Score', 'Precision Score', 'Recall Score', 'F1 Score', \
        'Youden J Score', 'MCC', 'Kappa Score']
models_report_out = pd.DataFrame(columns = cols)

for i in uclassLst:
    dsOut = classifiersLst[i][2]
    pred_test_out = classifiersLst[i][1].predict(Xo_orig[a[dsOut]])
    score = accuracy_score(Yo,pred_test_out)
    print('Accuracy Score for model : ', classifiersLst[i][0], ' is = ',score)
    Y_pred_rep_out[dsOut+'-'+classifiersLst[i][0]] = pred_test_out   # for submission file

    #  EVALUATE the MODEL
    print("Confusion Matrix : \n",confusion_matrix(Yo, pred_test_out))
    print("\n Classification report: \n",classification_report(Yo,pred_test_out),'\n')

    tmp2 = pd.Series({'DataSet': dsOut, \
                'Model': classifiersLst[i][0], \
                'Accuracy Score': metrics.accuracy_score(Yo, pred_test_out), \
                'ROC Score' : metrics.roc_auc_score(Yo, pred_test_out), \
                'Precision Score': metrics.precision_score(Yo, pred_test_out), \
                'Recall Score': metrics.recall_score(Yo, pred_test_out), \
                'F1 Score' : metrics.f1_score(Yo, pred_test_out), \
                'Youden J Score' : metrics.balanced_accuracy_score(Yo, pred_test_out), \
                'MCC': metrics.matthews_corrcoef(Yo, pred_test_out), \
                'Kappa Score':metrics.cohen_kappa_score(Yo, pred_test_out)})
    # models_report_out = models_report_out.append(tmp2, ignore_index = True)
    models_report_out = pd.concat([models_report_out, tmp2.to_frame().T], ignore_index=True)


Accuracy Score for model :  Logistic Regression  is =  0.6533455406948757
Confusion Matrix : 
 [[138222  73420]
 [    34    218]]

 Classification report: 
               precision    recall  f1-score   support

           0       1.00      0.65      0.79    211642
           1       0.00      0.87      0.01       252

    accuracy                           0.65    211894
   macro avg       0.50      0.76      0.40    211894
weighted avg       1.00      0.65      0.79    211894
 

Accuracy Score for model :  Gradient Boost  is =  0.0437294118757492
Confusion Matrix : 
 [[  9015 202627]
 [     1    251]]

 Classification report: 
               precision    recall  f1-score   support

           0       1.00      0.04      0.08    211642
           1       0.00      1.00      0.00       252

    accuracy                           0.04    211894
   macro avg       0.50      0.52      0.04    211894
weighted avg       1.00      0.04      0.08    211894
 

Accuracy Score for model :  Rando

## SUMMARY MODEL REPORT ON OUT-OF-SAMPLE DATA       <a id="5"></a>

In [ ]:
models_report_out

,DataSet,Model,Accuracy Score,ROC Score,Precision Score,Recall Score,F1 Score,Youden J Score,MCC,Kappa Score
0,DS_Original,Logistic Regression,0.653346,0.759086,0.00296,0.865079,0.005901,0.759086,0.037504,0.003539
1,DS_Original,Gradient Boost,0.043729,0.519314,0.001237,0.996032,0.002471,0.519314,0.006596,0.000096
2,DS_RFE,Random Forest,0.056538,0.521762,0.001244,0.988095,0.002485,0.521762,0.006559,0.00011
3,DS_Ada,Logistic Regression,0.653369,0.759098,0.002961,0.865079,0.005901,0.759098,0.037507,0.003539


#### Save the Reports to excel file.

In [ ]:
writer = pd.ExcelWriter(path + 'Models_Report.xlsx')
models_report_out.to_excel(writer, startrow=2, startcol=1)
models_report.to_excel(writer, startrow=len(uclassLst) + 5, startcol=1)
writer.close()


##               SUBMISSION OF PREDICTION

In [ ]:
df_original_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211894 entries, 0 to 211893
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           211894 non-null  int64  
 1   loan_amnt    211894 non-null  float64
 2   default_ind  211894 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 4.8 MB


In [ ]:
# Check for values match
df_original_sub.loc[:,'default_ind'].equals(pd.Series(Yo))

True

In [ ]:
# Get index for the Best Performing MODEL based on ROC score for Submission.
best = []
numx5 = models_report_out['ROC Score'].max()
best.append(models_report_out[models_report_out['ROC Score'] == numx5].index.values)

In [ ]:
submission = pd.DataFrame()
submission['Cust_ID']=df_original_sub['id']
submission['Loan_Amnt']=df_original_sub['loan_amnt']
submission['Default_Ind']= df_original_sub['default_ind']
submission['Default_Ind_Pred']= Y_pred_rep_out.iloc[:,best[0][0]]

submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211894 entries, 0 to 211893
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Cust_ID           211894 non-null  int64  
 1   Loan_Amnt         211894 non-null  float64
 2   Default_Ind       211894 non-null  int64  
 3   Default_Ind_Pred  211894 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 6.5 MB


#### Save Submission File as .csv

In [ ]:
submission.to_csv(path + 'Submission_Final.csv')

## ESTIMATED MONETARY IMPACT       <a id="6"></a>

Of the total defaulters of 252, the loan amount from [TRUE NEGATIVES = 218] predicted correctly by the final Logistic model is the monetary savings by the company.

In [ ]:
savings = submission.loc[(submission['Default_Ind']==1)  & (submission['Default_Ind_Pred']==1)]['Loan_Amnt'].sum()
print("The savings amount equals to : Rs.", savings)

The savings amount equals to : Rs. 3149675.0


##    CONCLUSION                <a id="7"></a>
### Based on ROC-AUC scores, Logistic Regression on DS_Ada dataset with 14 features, is the best model with ROC = 0.7591. After more performance tuning it should perform adequately and help in prediction of future data instances.